In [1]:
from copy import deepcopy
import math
from pathlib import Path

from arcgis.features import FeatureSet, FeatureLayerCollection, FeatureCollection
from arcgis.geometry import SpatialReference
from arcgis.gis import GIS, CreateServiceParameter
import pandas as pd
from tqdm.notebook import tqdm

from reach_tools import Reach

In [2]:
raw_dir_pth = Path.cwd().parent / f"data/raw/american_whitewater/"
url = 'https://adventure.maps.arcgis.com'
arcgis_client_id = "2MonB9785a7oRJEl"

In [3]:
gis = GIS(url, client_id=arcgis_client_id)

gis

Please sign in to your GIS and paste the code that is obtained below.
If a web browser does not automatically open, please navigate to the URL below yourself instead.
Opening web browser to navigate to: https://adventure.maps.arcgis.com/sharing/rest/oauth2/authorize?response_type=code&client_id=2MonB9785a7oRJEl&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&state=zRUgfa24DrGCByam3KtZAG1xbEmSDD&allow_verification=false


Enter code obtained on signing in using SAML:  ········


GIS @ https://adventure.maps.arcgis.com version:2025.1

In [11]:
overwrite = True

# if the service already exists, remove it
existing_lst = gis.content.search(service_name)

if len(existing_lst) > 0 and overwrite:
    gis.content.delete_items(existing_lst)
elif len(existing_lst) > 0:
    raise FileExistsError(f'Service "{service_name}" already exists in the ArcGIS Online organization.')

Recycle bin not enabled on this organization. Permanent delete parameter ignored.


In [12]:
## the service name and type to create
service_name = "API_CREATED_SERVICE"
service_type = "featureService"

## create the feature service
empty_service_item = gis.content.create_service(
    name = service_name,
    service_type = service_type,
)

fl_definition = {
    "type" : "Feature Layer",
    "name" : "API_CREATED_POINT_LAYER",
    "geometryType" : "esriGeometryPolyline", # esriGeometryPolygon, esriGeometryPolyline, esriGeometryPoint
    "fields" : [
        {
            "name" : "OBJECTID",
            "type" : "esriFieldTypeOID",
            "actualType" : "int",
            "alias" : "OBJECTID",
            "sqlType" : "sqlTypeInteger",
            "nullable" : False,
            "editable" : False
        },{
            "name": "INTEGER_FIELD",
            "type": "esriFieldTypeInteger",
            "actualType": "int",
            "alias": "Integer Field",
            "sqlType": "sqlTypeInteger",
            "nullable": True,
            "editable": True
        },{
          "name": "STRING_FIELD",
          "type": "esriFieldTypeString",
          "actualType": "nvarchar",
          "alias": "String Field",
          "sqlType": "sqlTypeNVarchar",
          "length": 25,
          "nullable": True,
          "editable": True
        },
        {
          "name": "DOUBLE_FIELD",
          "type": "esriFieldTypeDouble",
          "actualType": "float",
          "alias": "Decimal Field",
          "sqlType": "sqlTypeFloat",
          "nullable": True,
          "editable": True
        },
        {
          "name": "DATE_FIELD",
          "type": "esriFieldTypeDate",
          "alias": "Date Field",
          "sqlType": "sqlTypeOther",
          "nullable": True,
          "editable": True
        }
    ],
    "indexes" : [ # required index for Primary Key
        {
            "name" : "PK_IDX",
            "fields" : "OBJECTID",
            "isAscending" : True,
            "isUnique" : True,
            "description" : "clustered, unique, primary key"
        }
    ],
    "objectIdField" : "OBJECTID",
    "uniqueField" : { # a unique field is required
        "name" : "OBJECTID",
        "isSystemMaintained" : True
    },
    "spatialReference" : {
        "wkid" : 4326,
    }, 
}

## Create a FeatureLayerCollection object from the Item object
flc = FeatureLayerCollection.fromitem(empty_service_item)

## Update the JSON definition of the Service to include the layer
flc.manager.add_to_definition({"layers": [fl_definition]})

{'success': True, 'layers': [{'name': 'API_CREATED_POINT_LAYER', 'id': 0}]}

In [13]:
empty_service_item

<Item title:"API_CREATED_SERVICE" type:Feature Layer Collection owner:jmccune_personal>